<a href="https://colab.research.google.com/github/intellygentle/Alignedlayer-Testnet-Proof-/blob/main/SadTalker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#@title **setup（about 5 minutes）**
!apt-get update
!apt install software-properties-common -y
!add-apt-repository ppa:deadsnakes/ppa -y
!apt-get update
!apt-get install python3.8 python3.9 -y

# Try to fix any broken installations first
!apt --fix-broken install -y
!apt-get clean
!apt-get autoremove -y

# Install distutils and dev packages for python3.8
!apt-get install python3.8-distutils python3.8-dev python3.8-venv -y

# Ensure pip is installed for python3.8
!python3.8 -m ensurepip --default-pip
!python3.8 -m pip install --upgrade pip setuptools wheel

!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1
!python --version

print('Git clone project and install requirements...')
!git clone https://github.com/cedro3/SadTalker.git &> /dev/null
%cd SadTalker
!export PYTHONPATH=/content/SadTalker:$PYTHONPATH
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!apt update
!apt install ffmpeg -y &> /dev/null
!python3.8 -m pip install -r requirements.txt

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [2]:
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1

update-alternatives: error: alternative path /usr/bin/python3.8 doesn't exist
update-alternatives: error: alternative path /usr/bin/python3.9 doesn't exist


In [ ]:
#@title **download model（about 1 minute)**
print('Download pre-trained models...')
!rm -rf checkpoints
!bash scripts/download_models.sh

In [ ]:
#@title **inference for face**
image ='full3.png' #@param {type:"string"}
audio ='eluosi.wav' #@param {type:"string"}
source_image = 'examples/source_image/' + image
driven_audio = 'examples/driven_audio/' + audio

!python3.8 inference.py --driven_audio $driven_audio \
           --source_image $source_image \
           --result_dir ./results --enhancer gfpgan

In [ ]:
#@title **play movie**
import glob
from IPython.display import HTML
from base64 import b64encode
import os, sys

# get the last from results
mp4_name = sorted(glob.glob('./results/*.mp4'))[-1]

mp4 = open('{}'.format(mp4_name),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

print('Display animation: {}'.format(mp4_name), file=sys.stderr)
display(HTML("""
  <video width=256 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))

In [ ]:
#@title **inference for portrait**
image ='full3.png' #@param {type:"string"}
audio ='eluosi.wav' #@param {type:"string"}
source_image = 'examples/source_image/' + image
driven_audio = 'examples/driven_audio/' + audio

!python3.8 inference.py --driven_audio $driven_audio \
           --source_image $source_image \
           --result_dir ./results --still --preprocess full --enhancer gfpgan

In [ ]:
#@title **play movie**
import glob
from IPython.display import HTML
from base64 import b64encode
import os, sys

# get the last from results
mp4_name = sorted(glob.glob('./results/*.mp4'))[-1]

mp4 = open('{}'.format(mp4_name),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

print('Display animation: {}'.format(mp4_name), file=sys.stderr)
display(HTML("""
  <video width=256 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))

# Task
Generate and play an animated talking face and an animated talking portrait using the `full3.png` image and `eluosi.wav` audio.

## download_models

### Subtask:
Download pre-trained models required for the SadTalker project.


**Reasoning**:
The subtask requires downloading pre-trained models. The cell `eDw3_UN8K2xa` is designed to perform this action by first removing any existing 'checkpoints' directory and then executing the download script.



In [5]:
#@title **download model（about 1 minute)**
print('Download pre-trained models...')
!rm -rf checkpoints
!bash scripts/download_models.sh

Download pre-trained models...
--2026-01-21 13:40:50--  https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00109-model.pth.tar
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/569518584/ccc415aa-c6f4-47ee-8250-b10bf440ba62?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-21T14%3A27%3A07Z&rscd=attachment%3B+filename%3Dmapping_00109-model.pth.tar&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-01-21T13%3A27%3A02Z&ske=2026-01-21T14%3A27%3A07Z&sks=b&skv=2018-11-09&sig=Fav0eTxe6jCSv37jR0BDdtK%2FKrhKkZMs5j4hNFb%2BjPQ%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2OTAwNjQ1MSw

## inference_for_face

### Subtask:
Run face inference using a source image and driven audio.


**Reasoning**:
The subtask requires running face inference with specific image and audio files and saving the results to a specified directory. The provided code block will execute the inference script with all the necessary parameters, including the enhancer.



In [6]:
image ='full3.png' #@param {type:"string"}
audio ='eluosi.wav' #@param {type:"string"}
source_image = 'examples/source_image/' + image
driven_audio = 'examples/driven_audio/' + audio

!python3.8 inference.py --driven_audio $driven_audio \
           --source_image $source_image \
           --result_dir ./results --enhancer gfpgan

using safetensor as default
3DMM Extraction for source image
landmark Det:: 100% 1/1 [00:00<00:00, 16.05it/s]
3DMM Extraction In Video:: 100% 1/1 [00:00<00:00, 20.24it/s]
mel:: 100% 154/154 [00:00<00:00, 41273.02it/s]
audio2exp:: 100% 16/16 [00:00<00:00, 258.86it/s]
Face Renderer:: 100% 77/77 [00:42<00:00,  1.81it/s]
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (256, 259) to (256, 272) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
The generated video is named ./results/2026_01_21_13.44.12/full3##eluosi.mp4
face enhancer....
Face Enhancer:: 100% 154/154 [00:49<00:00,  3.13it/s]
The generated video is named ./results/2026_01_21_13.44.12/full3##eluosi_enhanced.mp4
The generated video is named: ./results/2026_01_21_13.44.12.mp4


**Reasoning**:
The previous step successfully generated the animated talking face video. The next logical step in the task is to play this generated video to verify the output. The `VfnFVzmybpfi` cell is designed to play the most recently generated video from the `results` directory.



In [7]:
import glob
from IPython.display import HTML
from base64 import b64encode
import os, sys

# get the last from results
mp4_name = sorted(glob.glob('./results/*.mp4'))[-1]

mp4 = open('{}'.format(mp4_name),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

print('Display animation: {}'.format(mp4_name), file=sys.stderr)
display(HTML("""
  <video width=256 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))

Display animation: ./results/2026_01_21_13.44.12.mp4


**Reasoning**:
The previous step successfully played the animated talking face video. The next logical step in the task is to generate an animated talking portrait using the specified image and audio. This is handled by the cell `lTses8gLVYoQ`.



In [8]:
image ='full3.png' #@param {type:"string"}
audio ='eluosi.wav' #@param {type:"string"}
source_image = 'examples/source_image/' + image
driven_audio = 'examples/driven_audio/' + audio

!python3.8 inference.py --driven_audio $driven_audio \
           --source_image $source_image \
           --result_dir ./results --still --preprocess full --enhancer gfpgan

using safetensor as default
3DMM Extraction for source image
landmark Det:: 100% 1/1 [00:00<00:00, 16.69it/s]
3DMM Extraction In Video:: 100% 1/1 [00:00<00:00, 56.57it/s]
mel:: 100% 154/154 [00:00<00:00, 42208.90it/s]
audio2exp:: 100% 16/16 [00:00<00:00, 264.90it/s]
Face Renderer:: 100% 77/77 [00:44<00:00,  1.74it/s]
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (256, 259) to (256, 272) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
The generated video is named ./results/2026_01_21_13.48.03/full3##eluosi.mp4
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
seamlessClone:: 100% 154/154 [00:23<00:00,  6.48it/s]
The generated video is named ./results/2026_01_21_13.48.03/full3

**Reasoning**:
The previous step successfully generated the animated talking portrait video. The next logical step is to play this generated video to verify the output. The `KOGCqUaib4CR` cell is designed to play the most recently generated video from the `results` directory.



In [9]:
import glob
from IPython.display import HTML
from base64 import b64encode
import os, sys

# get the last from results
mp4_name = sorted(glob.glob('./results/*.mp4'))[-1]

mp4 = open('{}'.format(mp4_name),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

print('Display animation: {}'.format(mp4_name), file=sys.stderr)
display(HTML("""
  <video width=256 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))

Display animation: ./results/2026_01_21_13.48.03.mp4


**Reasoning**:
All subtasks have been successfully completed, including generating and playing both the animated talking face and the animated talking portrait.



In [10]:
print('Task completed successfully!')

Task completed successfully!


## Summary:

### Data Analysis Key Findings

*   All necessary pre-trained models for the SadTalker project were successfully downloaded and saved to the `checkpoints` and `gfpgan/weights` directories.
*   An animated talking face was successfully generated using `full3.png` as the source image and `eluosi.wav` as the driven audio. The process included 3DMM extraction, audio-to-expression conversion, face rendering, and `gfpgan` enhancement, resulting in a video named `./results/2026_01_21_13.44.12.mp4`. This generated video was successfully played.
*   An animated talking portrait was successfully generated using the same `full3.png` and `eluosi.wav` inputs, with the addition of `--still` and `--preprocess full` arguments for portrait mode. This process involved seamless cloning and face enhancement, producing a video named `./results/2026_01_21_13.48.03.mp4`. This generated video was also successfully played.

### Insights or Next Steps

*   The SadTalker tool effectively generates both dynamic animated talking faces and static animated talking portraits from a single source image and audio file, showcasing its versatility for different visual requirements.
*   Further exploration could involve testing with diverse source images and audio files to evaluate the model's performance on various facial expressions, demographics, and speech characteristics, or experimenting with other enhancement options beyond `gfpgan`.
